In [ ]:
# Import Qiskit libraries
import qiskit
import qiskit_aer.noise as noise
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, QiskitError
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session, Batch
from qiskit.circuit import Parameter
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.primitives import StatevectorSampler
from qiskit.providers import JobError
from qiskit.visualization import plot_error_map

# Save an IBM Quantum account and set it as your default account.
QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    instance='ibm-q/open/main',
    token="[your token]",
    set_as_default=True,
    overwrite=True,
)

# Load saved credentials
service = QiskitRuntimeService()

In [ ]:
# Import python libraries
import math, json
import numpy as np
from numpy import pi
from numpy.linalg import eigvals
from numpy.fft import fft, rfft
from numpy.polynomial import Polynomial
from scipy.interpolate import CubicSpline
from scipy.interpolate import UnivariateSpline
from scipy.signal import find_peaks
from scipy.optimize import curve_fit, minimize, minimize_scalar
from pybaselines import Baseline
from scipy.stats import skew
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib import colors
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter
from concurrent.futures import ThreadPoolExecutor, as_completed
import sys, os, threading, time, random

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('text.latex', preamble=r'\usepackage{amsmath,amsfonts,amssymb}')

# Define a custom formatter for the y-ticks to display 10^(-n+1), and 1 for 10^(-0+1)
def format_exponent(value, tick_number):
    exponent = int(value) + 1  # Increment the exponent by 1
    if exponent == 0:
        return r'$1$'  # Display 10^0 as 1
    else:
        return r'$10^{-%d}$' % exponent

# Calculate initial gap
def gap_ini_estimator(Nc):

    # set the initial guess of energy gap
    gap_ini = 2*(1-(1-1/Nc)*g)

    return gap_ini

# Calculate exact gap
def gap_exact_estimator(Nc):

    # Define the identity and Pauli matrices
    s0 = np.matrix([[1,0],[0,1]])
    s1 = np.matrix([[0,1],[1,0]])
    s2 = np.matrix([[0,-1j],[1j,0]])
    s3 = np.matrix([[1,0],[0,-1]])
    
    # Define the model Hamitonian
    Ham = np.kron(s1,s0) + np.kron(s0,s1) + g * np.kron(s3,s3)
    if Nc >= 3:
        for j in range(3,Nc+1):  
            Ide = s0
            for k in range(j-3):
                Ide = np.kron(Ide,s0)
            Ham = np.kron(Ham,s0) + np.kron(np.kron(Ide,s0),s1) + g * np.kron(np.kron(Ide,s3),s3)

    # Calculate the exact energy gap
    evals = eigvals(Ham)
    idx = evals.argsort()[::1]
    evals = evals[idx]
    gap_exact = evals[1] - evals[0]

    return gap_exact

# Define objective function for curve fitting of the Lorentzian or Gaussian function
def fit_Lorentzian (x, a0, a1, a2, a3):
    return a0 + a1**2 / ((x - a2)**2 + a3**2)

def fit_Gaussian (x, a0, a1, a2, a3):
    return a0 + a1**2 * np.exp(- np.log(2) * (x - a2)**2 / a3**2)

In [ ]:
# Set up parameters
Nc = 5  # number of spins: N (= 5, 10, 15, 20)
g = 0.4  # dimensionless Ising coupling: J/h (= 0.2, 0.4, 0.6, 0.8)
eta = 0.3  # dimensionless broadening: \eta/h (= 0.02, 0.1, 0.2, 0.3)
dw = 0.25*eta  # dimensionless unit of discrete frequency: \delta\omega / h
L = 2*math.ceil(5.0/dw)  # count of Fourier components: L
dt = 2*pi/L/dw  # dimensionless unit of discrete time: h \delta t
M = 15
shots = 1024  # count of measurement shots

# Define the bit string (00...0) as a reference for the initial state
InQ = ''.join(['0' for _ in range(Nc)])

# Define discrete frequencies
frequency = [(j-L/2)*dw for j in range(L+1)]
with open('data_frequency_N5', 'w') as f:
    f.write(json.dumps(frequency))

# Calculate initial gap
gap_ini = gap_ini_estimator(Nc)

# Calculate exact gap
gap_exact = gap_exact_estimator(Nc)

# Print preliminary data
print("N = %s, J/h = %.2f, eta/h = %.2f, M = %s" % (Nc,g,eta,M))
print("Initial gap = %.6f" % gap_ini)
print("Exact gap = %.6f" % gap_exact)

In [ ]:
# Set real device backend
backend = service.backend("ibm_sherbrooke")

# Build noise model from backend calibration data
noise_model = NoiseModel.from_backend(backend)
coupling_map = backend.configuration().coupling_map
basis_gates = noise_model.basis_gates

In [ ]:
# Get backend properties
properties = backend.properties()

# Print out the calibration data
for qubit in properties.qubits:
    print(f"Qubit {properties.qubits.index(qubit)}:")
    for item in qubit:
        print(f"  {item.name}: {item.value} {item.unit}")

for gate in properties.gates:
    print(f"Gate {gate.name} between qubits {gate.qubits}:")
    for param in gate.parameters:
        print(f"  {param.name}: {param.value} {param.unit}")

In [ ]:
# Initialize a set of quantum/classical registers
qreg = QuantumRegister(Nc, 'q')
creg = ClassicalRegister(Nc, 'c')
qc = QuantumCircuit(qreg, creg)

i = 1
j = 1

# Set up the unitary for trial state
for n in range(Nc):
    qc.ry(0.3 * pi, qreg[n])
    
    # Time evolution: Iterate Trotter steps
    for m in range(M):

        # Apply a sequence of unitary gates for time evolution
        for n in range(Nc):
            qc.rx(-2 * j * (-1)**i * dt / M, qreg[n])
                
        for n in range(Nc - 1):
            qc.rzz(-2 * g * j * (-1)**i * dt / M, qreg[n], qreg[n+1])
                    
    # Set up the inverse unitary for trial state
    for n in range(Nc):
        qc.ry(-0.3 * pi, qreg[n])
        
    # Measure qubits
    qc.measure(qreg, creg)

# Define your quantum circuit `qc` and backend
transpiled_circuit = transpile(qc, backend, optimization_level=2)

# Print the chosen layout
print("Best Qubit Layout (Logical-to-Physical Mapping):")
print(transpiled_circuit.layout)

In [ ]:
# Define main function
def main_function(theta, session):

    print('In progress...', end="\r")
    
    global sim_type, filter_type, gap_ini
    
    initial_layout = [113,114,115,116,117]
    max_retries = 3  # Number of retries
    retry_delay = 60  # Delay between retries in seconds

    # Define parameters
    Para_i = Parameter('Parameter_i')
    Para_j = Parameter('Parameter_j')

    # Initialize quantum/classical registers
    qreg = QuantumRegister(Nc, 'q')
    creg = ClassicalRegister(Nc, 'c')
    qc = QuantumCircuit(qreg, creg)
    
    # Set up the unitary for trial state
    for n in range(Nc):
        qc.ry(theta * np.pi, qreg[n])
    
    # Time evolution: Iterate Trotter steps
    for m in range(M):
        for n in range(Nc):
            qc.rx(-2 * Para_j * (-1)**Para_i * dt / M, qreg[n])
                
        for n in range(Nc - 1):
            qc.rzz(-2 * g * Para_j * (-1)**Para_i * dt / M, qreg[n], qreg[n + 1])
                    
    # Inverse unitary for trial state
    for n in range(Nc):
        qc.ry(-theta * np.pi, qreg[n])
        
    # Measure qubits
    qc.measure(qreg, creg)

    # Allocate lists
    counts_batch = []
    index_mapping = []
    
    if sim_type == 1:

        # Prepare all circuits for causal (i=0) and anti-causal (i=1) processes
        bound_circuits = []
        for i in range(2):
            for j in range(L):
                bound_circuit = qc.assign_parameters({Para_i: i, Para_j: j})
                bound_circuits.append(bound_circuit)
                index_mapping.append((i, j))

        # Set simulator
        simulator = AerSimulator()
    
        # Run all bound circuits in one job
        job = simulator.run(bound_circuits, shots=shots)
        results = job.result()
    
        # Extract measurement counts for the circuits
        for idx, (i, j) in enumerate(index_mapping):
            counts = results.get_counts(idx)
            counts_batch.append(counts)
    
    if sim_type == 2:
        
        # Transpile the circuit
        qc_trans = transpile(qc, backend=backend, 
                             optimization_level=3, 
                             initial_layout=initial_layout)

        # Prepare all circuits for causal (i=0) and anti-causal (i=1) processes
        bound_circuits = []
        for i in range(2):
            for j in range(L):
                bound_circuit = qc_trans.assign_parameters({Para_i: i, Para_j: j})
                bound_circuits.append(bound_circuit)
                index_mapping.append((i, j))

        # Set simulator
        simulator = AerSimulator()
    
        # Run all bound circuits in one job
        job = simulator.run(bound_circuits, shots=shots, 
                            noise_model=noise_model, 
                            coupling_map=coupling_map,
                            basis_gates=basis_gates)
    
        results = job.result()
    
        # Extract measurement counts for the circuits
        for idx, (i, j) in enumerate(index_mapping):
            counts = results.get_counts(idx)
            counts_batch.append(counts)

    if sim_type == 3:

        # Transpile the circuit
        qc_trans = transpile(qc, backend=backend, 
                             optimization_level=3, 
                             initial_layout=initial_layout)

        # Prepare all circuits for causal (i=0) and anti-causal (i=1) processes
        bound_circuits = []
        for i in range(2):
            for j in range(L):
                bound_circuit = qc_trans.assign_parameters({Para_i: i, Para_j: j})
                bound_circuits.append(bound_circuit)
                index_mapping.append((i, j))

        # Set sampler
        sampler = Sampler(mode=session)
            
        # Initialize retry loop for job execution
        for attempt in range(max_retries):
            try:
                # Run all bound circuits in one job
                job = sampler.run(bound_circuits, shots=shots)
                results = job.result(timeout = 6 * 3600)
                break  # Exit loop if job succeeds
            except Exception as e:
                print(f"Attempt {attempt + 1} failed with error: {e}")
                if attempt + 1 < max_retries:
                    print(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    raise RuntimeError("Max retries reached. Job failed.")
        
        # Extract measurement counts for the circuits
        for idx, (i, j) in enumerate(index_mapping):
            counts = results[idx].data.c.get_counts()
            counts_batch.append(counts)

    # Initialize arrays for propagators
    propagators = {0: [], 1: []}

    # Loop through the results and postprocess them
    for idx, (i, j) in enumerate(index_mapping):
        counts = counts_batch[idx]

        # Apply the filter depending on the filter type
        filter_func = np.exp(-eta * j * dt) if filter_type == 1 else np.exp(-(eta * j * dt)**2 / (4 * np.log(2)))

        # Calculate the propagator value
        propagator_value = dt * filter_func * counts.get(InQ, 0) / shots
        propagators[i].append(propagator_value)

    # Flatten the arrays for Fourier transforms
    propagator_0 = np.ravel(propagators[0])
    propagator_1 = np.ravel(propagators[1])

    # Compute the Fourier transforms (discrete)
    spectral_function_0r = np.real(np.fft.ifft(propagator_0)) * L / (2.0 * np.pi)
    spectral_function_1r = np.append(spectral_function_0r[int(L/2):L], spectral_function_0r[0:int(L/2)+1])

    spectral_function_0a = np.real(np.fft.fft(propagator_1)) / (2.0 * np.pi)
    spectral_function_1a = np.append(spectral_function_0a[int(L/2):L], spectral_function_0a[0:int(L/2)+1])
  
    # Find the spectral function via discrete fast Fourier transform
    spectral_function_2 = spectral_function_1r + spectral_function_1a

    # Convert to array
    spectral_function = np.array(spectral_function_2)
    
    # Interpolate data
    spline_obj = CubicSpline(frequency, spectral_function)
    frequency_fine = np.linspace(0, 4, 50 * len(frequency))
    spectral_function_fine = spline_obj(frequency_fine)

    # Gap estimation (original)
    peaks, _ = find_peaks(spectral_function_fine)
    if peaks.size > 0:
        max_index_org = peaks[np.abs(frequency_fine[peaks] - gap_ini).argmin()]
        gap_est_org = frequency_fine[max_index_org]
    else:
        gap_est_org = 1e6
    
    # ALS baseline correction
    baseline_obj = Baseline()
    background, params = baseline_obj.asls(spectral_function_fine, lam=lam, p=asym)
    spectral_function_modified = spectral_function_fine - background
    
    mask = (frequency_fine >= gap_ini-eta) & (frequency_fine <= gap_ini+eta)
    mean = np.mean(spectral_function_modified * mask)
    std_dev = np.std(spectral_function_modified * mask)
    threshold = mean + 1 * std_dev
    peaks, _ = find_peaks(spectral_function_modified * mask, height=threshold)
    if peaks.size > 0:
        max_index = peaks[np.abs(frequency_fine[peaks] - gap_ini).argmin()]
        gap_est = frequency_fine[max_index]
    else:
        gap_est = 1e6
    
    # Cost function
    cost_function = 1 / spectral_function_modified[max_index] if max_index is not None else None
    
    return cost_function, gap_est_org, gap_est, spectral_function

In [ ]:
# Focus on trial-state optimization

# Print header
print("Simulation type: ideal")
print("Optimization method: bounded")
print('{0:4s}  {1:9s}      {2:9s}      {3:9s}    {4:9s}    {5:9s}      {6:9s}'\
      .format('Iteration','theta/pi','cost_func','gap_est_org','gap_err_org','gap_est','gap_err'))

# Set parameters
sim_type = 1 # Simulation type: real device
filter_type = 1 # Filter type (1: Lorentzian, 2: Gaussian)
bounds = (0,0.5) # exclude unfavored regime from (0,0.5)
options = {'xatol':1e-6,'maxiter':1000,'disp': True}
#lam = 1e13 if filter_type == 1 else 1e9  # smoothing parameter in ALS baseline correction
lam = 1e0 # smoothing parameter in ALS baseline correction
asym = 1e-2 # asymmetry parameter in ALS baseline correction
theta_0 = None
Nfeval = 0
data_spectral = []
data_iteration = []

with open('data_frequency_N5', 'r') as f:
    frequency = np.array(json.load(f))

# Define the objective function
def objective_function(theta):
    
    global sim_type, filter_type, gap_exact, gap_ini, eta, data, Nfeval, theta_0
    
    # Check for stopping condition
    if theta_0 is not None and abs(theta - theta_0)/theta_0 < 1e-3:
        print(f'Stopping optimization early at iteration {Nfeval}, theta change is below 1e-3.')
        raise ValueError("Optimization stopped due to small theta change.")

    # Evaluations
    cost_function, gap_est_org, gap_est, spectral_function = main_function(theta, 0)
    gap_err_org = abs(gap_est_org - gap_exact)/gap_exact
    gap_err = abs(gap_est - gap_exact)/gap_exact

    # If gap_est_org is outside the frequency window, return a large penalty cost
    if not (gap_ini - eta <= gap_est_org <= gap_ini + eta):
        print(f'gap_est_org={gap_est_org} outside frequency window, returning large penalty.')
        return 1e50  # Penalize by returning large number
    
    # Append current data
    Nfeval += 1
    data_spectral.append(spectral_function.tolist())
    data_iteration.append([Nfeval, theta, cost_function, gap_est_org, gap_err_org, gap_est, gap_err])
    
    # Print data
    print('     {0:4d}  {1:3.6e}   {2:3.6e}   {3:3.6e}   {4:3.6e}   {5:3.6e}   {6:3.6e}'\
          .format(Nfeval, theta, cost_function, gap_est_org, gap_err_org, gap_est, gap_err))

    # Set file names
    if filter_type == 1:
        # Lorentzian file names
        filename_spectral = 'data_spectral_ideal_N%s_g%.2f_eta%.2f_M%s_FL_BD' % (int(Nc), g, eta, int(M))
        filename_iteration = 'data_iteration_ideal_N%s_g%.2f_eta%.2f_M%s_FL_BD' % (int(Nc), g, eta, int(M))
    else:
        # Gaussian file names
        filename_spectral = 'data_spectral_ideal_N%s_g%.2f_eta%.2f_M%s_FG_BD' % (int(Nc), g, eta, int(M))
        filename_iteration = 'data_iteration_ideal_N%s_g%.2f_eta%.2f_M%s_FG_BD' % (int(Nc), g, eta, int(M))

    # Write data
    with open(filename_spectral, 'w') as f:
        f.write(json.dumps(data_spectral))

    with open(filename_iteration, 'w') as f:
        f.write(json.dumps(data_iteration))
    
    # Update old theta
    theta_0 = theta
    
    return cost_function

# Run optimization
try:
    result = minimize_scalar(
        objective_function, 
        method='bounded', 
        bounds=bounds, 
        options=options
        )
except ValueError as e:
    print(f"Optimization stopped: {e}")

In [ ]:
# Focus on trial-state optimization

# Print header
print("Simulation type: noisy")
print("Device backend: ibm_sherbrooke")
print("Optimization method: bounded")
print('{0:4s}  {1:9s}      {2:9s}      {3:9s}    {4:9s}    {5:9s}      {6:9s}'\
      .format('Iteration','theta/pi','cost_func','gap_est_org','gap_err_org','gap_est','gap_err'))

# Set parameters
sim_type = 2 # Simulation type: real device
filter_type = 1 # Filter type (1: Lorentzian, 2: Gaussian)
bounds = (0,0.5) # exclude unfavored regime from (0,0.5)
options = {'xatol':1e-6,'maxiter':1000,'disp': True}
#lam = 1e13 if filter_type == 1 else 1e9  # smoothing parameter in ALS baseline correction
lam = 1e0 # smoothing parameter in ALS baseline correction
asym = 1e-2  # asymmetry parameter in ALS baseline correction
theta_0 = None
Nfeval = 0
data_spectral = []
data_iteration = []

with open('data_frequency_N5', 'r') as f:
    frequency = np.array(json.load(f))

# Define the objective function
def objective_function(theta):
    
    global sim_type, filter_type, gap_exact, gap_ini, eta, data, Nfeval, theta_0
    
    # Check for stopping condition
    if theta_0 is not None and abs(theta - theta_0)/theta_0 < 1e-3:
        print(f'Stopping optimization early at iteration {Nfeval}, theta change is below 1e-3.')
        raise ValueError("Optimization stopped due to small theta change.")

    # Evaluations
    cost_function, gap_est_org, gap_est, spectral_function = main_function(theta, 0)
    gap_err_org = abs(gap_est_org - gap_exact)/gap_exact
    gap_err = abs(gap_est - gap_exact)/gap_exact

    # Append current data
    Nfeval += 1
    data_spectral.append(spectral_function.tolist())
    data_iteration.append([Nfeval, theta, cost_function, gap_est_org, gap_err_org, gap_est, gap_err])
    
    # Print data
    print('     {0:4d}  {1:3.6e}   {2:3.6e}   {3:3.6e}   {4:3.6e}   {5:3.6e}   {6:3.6e}'\
          .format(Nfeval, theta, cost_function, gap_est_org, gap_err_org, gap_est, gap_err))

    # Set file names
    if filter_type == 1:
        # Lorentzian file names
        filename_spectral = 'data_spectral_noisy_N%s_g%.2f_eta%.2f_M%s_FL_BD' % (int(Nc), g, eta, int(M))
        filename_iteration = 'data_iteration_noisy_N%s_g%.2f_eta%.2f_M%s_FL_BD' % (int(Nc), g, eta, int(M))
    else:
        # Gaussian file names
        filename_spectral = 'data_spectral_noisy_N%s_g%.2f_eta%.2f_M%s_FG_BD' % (int(Nc), g, eta, int(M))
        filename_iteration = 'data_iteration_noisy_N%s_g%.2f_eta%.2f_M%s_FG_BD' % (int(Nc), g, eta, int(M))

    # Write data
    with open(filename_spectral, 'w') as f:
        f.write(json.dumps(data_spectral))

    with open(filename_iteration, 'w') as f:
        f.write(json.dumps(data_iteration))
    
    # Update old theta
    theta_0 = theta
    
    return cost_function

# Run optimization
try:
    result = minimize_scalar(
        objective_function, 
        method='bounded', 
        bounds=bounds, 
        options=options
        )
except ValueError as e:
    print(f"Optimization stopped: {e}")

In [ ]:
# Focus on trial-state optimization

# Print header
print("Simulation type: real device")
print("Device backend: ibm_sherbrooke")
print("Optimization method: bounded")
print('{0:4s}  {1:9s}      {2:9s}      {3:9s}    {4:9s}    {5:9s}      {6:9s}'\
      .format('Iteration','theta/pi','cost_func','gap_est_org','gap_err_org','gap_est','gap_err'))

# Set parameters
sim_type = 3 # Simulation type: real device
filter_type = 1 # Filter type (1: Lorentzian, 2: Gaussian)
bounds = (0,0.5) # exclude unfavored regime from (0,0.5)
options = {'xatol':1e-6,'maxiter':1000,'disp': True}
lam = 1e0  # smoothing parameter in ALS baseline correction
asym = 1e-2  # asymmetry parameter in ALS baseline correction
theta_0 = None
Nfeval = 0
data_spectral = []
data_iteration = []

with open('data_frequency_N5', 'r') as f:
    frequency = np.array(json.load(f))

# Define the objective function
def objective_function(theta, session):
    
    global sim_type, filter_type, gap_exact, gap_ini, eta, data, Nfeval, theta_0
    
    # Check for stopping condition
    if theta_0 is not None and abs(theta - theta_0)/theta_0 < 1e-3:
        print(f'Stopping optimization early at iteration {Nfeval}, theta change is below 1e-3.')
        raise ValueError("Optimization stopped due to small theta change.")

    # Evaluations
    cost_function, gap_est_org, gap_est, spectral_function = main_function(theta, session)
    gap_err_org = abs(gap_est_org - gap_exact)/gap_exact
    gap_err = abs(gap_est - gap_exact)/gap_exact

    # Append current data
    Nfeval += 1
    data_spectral.append(spectral_function.tolist())
    data_iteration.append([Nfeval, theta, cost_function, gap_est_org, gap_err_org, gap_est, gap_err])
    
    # Print data
    print('     {0:4d}  {1:3.6e}   {2:3.6e}   {3:3.6e}   {4:3.6e}   {5:3.6e}   {6:3.6e}'\
          .format(Nfeval, theta, cost_function, gap_est_org, gap_err_org, gap_est, gap_err))

    # Set file names
    if filter_type == 1:
        # Lorentzian file names
        filename_spectral = 'data_spectral_real_N%s_g%.2f_eta%.2f_M%s_FL_BD' % (int(Nc), g, eta, int(M))
        filename_iteration = 'data_iteration_real_N%s_g%.2f_eta%.2f_M%s_FL_BD' % (int(Nc), g, eta, int(M))
    else:
        # Gaussian file names
        filename_spectral = 'data_spectral_real_N%s_g%.2f_eta%.2f_M%s_FG_BD' % (int(Nc), g, eta, int(M))
        filename_iteration = 'data_iteration_real_N%s_g%.2f_eta%.2f_M%s_FG_BD' % (int(Nc), g, eta, int(M))

    # Write data
    with open(filename_spectral, 'w') as f:
        f.write(json.dumps(data_spectral))

    with open(filename_iteration, 'w') as f:
        f.write(json.dumps(data_iteration))
    
    # Update old theta
    theta_0 = theta
    
    return cost_function

# Run optimization in session mode
with Session(backend=backend) as session:
    try:
        result = minimize_scalar(
            objective_function, 
            method='bounded', 
            bounds=bounds, 
            options=options,
            args=(session,)
        )
    except ValueError as e:
        print(f"Optimization stopped: {e}")

In [ ]:
# Focus on trial-state optimization

# Print header
print("Simulation type: real device")
print("Device backend: ibm_sherbrooke")
print("Optimization method: bounded")
print('{0:4s}  {1:9s}      {2:9s}      {3:9s}    {4:9s}    {5:9s}      {6:9s}'\
      .format('Iteration','theta/pi','cost_func','gap_est_org','gap_err_org','gap_est','gap_err'))

# Set parameters
sim_type = 3 # Simulation type: real device
filter_type = 2 # Filter type (1: Lorentzian, 2: Gaussian)
bounds = (0,0.5) # exclude unfavored regime from (0,0.5)
options = {'xatol':1e-6,'maxiter':1000,'disp': True}
lam = 1e-2  # smoothing parameter in ALS baseline correction
asym = 1e-2  # asymmetry parameter in ALS baseline correction
theta_0 = None
Nfeval = 0
data_spectral = []
data_iteration = []

with open('data_frequency_N5', 'r') as f:
    frequency = np.array(json.load(f))

# Define the objective function
def objective_function(theta, session):
    
    global sim_type, filter_type, gap_exact, gap_ini, eta, data, Nfeval, theta_0
    
    # Check for stopping condition
    if theta_0 is not None and abs(theta - theta_0)/theta_0 < 1e-3:
        print(f'Stopping optimization early at iteration {Nfeval}, theta change is below 1e-3.')
        raise ValueError("Optimization stopped due to small theta change.")

    # Evaluations
    cost_function, gap_est_org, gap_est, spectral_function = main_function(theta, session)
    gap_err_org = abs(gap_est_org - gap_exact)/gap_exact
    gap_err = abs(gap_est - gap_exact)/gap_exact

    # Append current data
    Nfeval += 1
    data_spectral.append(spectral_function.tolist())
    data_iteration.append([Nfeval, theta, cost_function, gap_est_org, gap_err_org, gap_est, gap_err])
    
    # Print data
    print('     {0:4d}  {1:3.6e}   {2:3.6e}   {3:3.6e}   {4:3.6e}   {5:3.6e}   {6:3.6e}'\
          .format(Nfeval, theta, cost_function, gap_est_org, gap_err_org, gap_est, gap_err))

    # Set file names
    if filter_type == 1:
        # Lorentzian file names
        filename_spectral = 'data_spectral_real_N%s_g%.2f_eta%.2f_M%s_FL_BD' % (int(Nc), g, eta, int(M))
        filename_iteration = 'data_iteration_real_N%s_g%.2f_eta%.2f_M%s_FL_BD' % (int(Nc), g, eta, int(M))
    else:
        # Gaussian file names
        filename_spectral = 'data_spectral_real_N%s_g%.2f_eta%.2f_M%s_FG_BD' % (int(Nc), g, eta, int(M))
        filename_iteration = 'data_iteration_real_N%s_g%.2f_eta%.2f_M%s_FG_BD' % (int(Nc), g, eta, int(M))

    # Write data
    with open(filename_spectral, 'w') as f:
        f.write(json.dumps(data_spectral))

    with open(filename_iteration, 'w') as f:
        f.write(json.dumps(data_iteration))
    
    # Update old theta
    theta_0 = theta
    
    return cost_function

# Run optimization in session mode
with Session(backend=backend) as session:
    try:
        result = minimize_scalar(
            objective_function, 
            method='bounded', 
            bounds=bounds, 
            options=options,
            args=(session,)
        )
    except ValueError as e:
        print(f"Optimization stopped: {e}")

In [ ]:
# Convergence test

# Print start message
print("Simulation type: ideal")
print("Task: convergence test")
print('{0:4s}  {1:9s}      {2:9s}    {3:9s}    {4:9s}      {5:9s}'\
      .format('M','cost_func','gap_est_org','gap_err_org','gap_est','gap_err'))

# Set parameters
sim_type = 1 # Simulation type: ideal
filter_type = 2 # Filter type (1: Lorentzian, 2: Gaussian)
theta = 0.3
lam = 1e0  # smoothing parameter in ALS baseline correction
asym = 1e-2  # asymmetry parameter in ALS baseline correction
data_spectral = []
data_iteration = []

with open('data_frequency_N5', 'r') as f:
    frequency = np.array(json.load(f))

for n in range(19):

    # Set Trotter depth
    M = 2*n + 3
    
    # Evaluations
    cost_function, gap_est_org, gap_est, spectral_function = main_function(theta, 0)
    gap_err_org = abs(gap_est_org - gap_exact)/gap_exact
    gap_err = abs(gap_est - gap_exact)/gap_exact

    # Append current data
    data_spectral.append(spectral_function.tolist())
    data_iteration.append([M, theta, cost_function, gap_est_org, gap_err_org, gap_est, gap_err])
    
    # Print data
    print('{0:2d}  {1:3.6e}   {2:3.6e}   {3:3.6e}   {4:3.6e}   {5:3.6e}'\
          .format(M, cost_function, gap_est_org, gap_err_org, gap_est, gap_err))

    # Set file names
    if filter_type == 1:
        # Lorentzian file names
        filename_spectral = 'data_spectral_ideal_N%s_g%.2f_eta%.2f_theta%.2f_FL_CV' % (int(Nc), g, eta, theta)
        filename_iteration = 'data_iteration_ideal_N%s_g%.2f_eta%.2f_theta%.2f_FL_CV' % (int(Nc), g, eta, theta)
    else:
        # Gaussian file names
        filename_spectral = 'data_spectral_ideal_N%s_g%.2f_eta%.2f_theta%.2f_FG_CV' % (int(Nc), g, eta, theta)
        filename_iteration = 'data_iteration_ideal_N%s_g%.2f_eta%.2f_theta%.2f_FG_CV' % (int(Nc), g, eta, theta)

    # Write data
    with open(filename_spectral, 'w') as f:
        f.write(json.dumps(data_spectral))

    with open(filename_iteration, 'w') as f:
        f.write(json.dumps(data_iteration))

# End message
print("Job completed")

In [ ]:
# Convergence test

# Print start message
print("Simulation type: noisy")
print("Device backend: ibm_sherbrooke")
print("Task: convergence test")
print('{0:4s}  {1:9s}      {2:9s}    {3:9s}    {4:9s}      {5:9s}'\
      .format('M','cost_func','gap_est_org','gap_err_org','gap_est','gap_err'))

# Set parameters
sim_type = 2 # Simulation type: ideal
filter_type = 2 # Filter type (1: Lorentzian, 2: Gaussian)
theta = 0.3
lam = 1e0  # smoothing parameter in ALS baseline correction
asym = 1e-2  # asymmetry parameter in ALS baseline correction
data_spectral = []
data_iteration = []

with open('data_frequency_N5', 'r') as f:
    frequency = np.array(json.load(f))

for n in range(19):

    # Set Trotter depth
    M = 2*n + 3
    
    # Evaluations
    cost_function, gap_est_org, gap_est, spectral_function = main_function(theta, 0)
    gap_err_org = abs(gap_est_org - gap_exact)/gap_exact
    gap_err = abs(gap_est - gap_exact)/gap_exact

    # Append current data
    data_spectral.append(spectral_function.tolist())
    data_iteration.append([M, theta, cost_function, gap_est_org, gap_err_org, gap_est, gap_err])
    
    # Print data
    print('{0:2d}  {1:3.6e}   {2:3.6e}   {3:3.6e}   {4:3.6e}   {5:3.6e}'\
          .format(M, cost_function, gap_est_org, gap_err_org, gap_est, gap_err))

    # Set file names
    if filter_type == 1:
        # Lorentzian file names
        filename_spectral = 'data_spectral_noisy_N%s_g%.2f_eta%.2f_theta%.2f_FL_CV' % (int(Nc), g, eta, theta)
        filename_iteration = 'data_iteration_noisy_N%s_g%.2f_eta%.2f_theta%.2f_FL_CV' % (int(Nc), g, eta, theta)
    else:
        # Gaussian file names
        filename_spectral = 'data_spectral_noisy_N%s_g%.2f_eta%.2f_theta%.2f_FG_CV' % (int(Nc), g, eta, theta)
        filename_iteration = 'data_iteration_noisy_N%s_g%.2f_eta%.2f_theta%.2f_FG_CV' % (int(Nc), g, eta, theta)

    # Write data
    with open(filename_spectral, 'w') as f:
        f.write(json.dumps(data_spectral))

    with open(filename_iteration, 'w') as f:
        f.write(json.dumps(data_iteration))

# End message
print("Job completed")

In [ ]:
# Convergence test

# Print start message
print("Simulation type: real device")
print("Device backend: ibm_sherbrooke")
print("Task: convergence test")
print('{0:4s}  {1:9s}      {2:9s}    {3:9s}    {4:9s}      {5:9s}'\
      .format('M','cost_func','gap_est_org','gap_err_org','gap_est','gap_err'))

# Set parameters
sim_type = 3 # Simulation type: real device
filter_type = 2 # Filter type (1: Lorentzian, 2: Gaussian)
theta = 0.3
lam = 1e0  # smoothing parameter in ALS baseline correction
asym = 1e-2  # asymmetry parameter in ALS baseline correction
data_spectral = []
data_iteration = []

with open('data_frequency_N5', 'r') as f:
    frequency = np.array(json.load(f))

# Run jobs in session mode
with Session(backend=backend) as session:
    
    for n in range(19):

        # Set Trotter depth
        M = 2*n + 3
    
        # Evaluations
        cost_function, gap_est_org, gap_est, spectral_function = main_function(theta, session)
        gap_err_org = abs(gap_est_org - gap_exact)/gap_exact
        gap_err = abs(gap_est - gap_exact)/gap_exact

        # Append current data
        data_spectral.append(spectral_function.tolist())
        data_iteration.append([M, theta, cost_function, gap_est_org, gap_err_org, gap_est, gap_err])
    
        # Print data
        print('{0:2d}  {1:3.6e}   {2:3.6e}   {3:3.6e}   {4:3.6e}   {5:3.6e}'\
              .format(M, cost_function, gap_est_org, gap_err_org, gap_est, gap_err))

        # Set file names
        if filter_type == 1:
            # Lorentzian file names
            filename_spectral = 'data_spectral_real_N%s_g%.2f_eta%.2f_theta%.2f_FL_CV' % (int(Nc), g, eta, theta)
            filename_iteration = 'data_iteration_real_N%s_g%.2f_eta%.2f_theta%.2f_FL_CV' % (int(Nc), g, eta, theta)
        else:
            # Gaussian file names
            filename_spectral = 'data_spectral_real_N%s_g%.2f_eta%.2f_theta%.2f_FG_CV' % (int(Nc), g, eta, theta)
            filename_iteration = 'data_iteration_real_N%s_g%.2f_eta%.2f_theta%.2f_FG_CV' % (int(Nc), g, eta, theta)

        # Write data
        with open(filename_spectral, 'w') as f:
            f.write(json.dumps(data_spectral))

        with open(filename_iteration, 'w') as f:
            f.write(json.dumps(data_iteration))

# End message
print("Job completed")